In [1]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [2]:
def process_data(file):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    return np.array(coord)

In [3]:
def read_directory(directory, sample_rate):
    point_array_all = []
    land_mark_array_all=[]

    for filename in sorted(os.listdir(directory)):

        if filename.endswith(".pts"): 
            path=os.path.join(directory, filename)
            #print("Processing:",path)
            points = open(path)
            point_array = process_data(points)

            point_array_sampled = []
            
            #sampling
            for i in range(0,len(point_array),sample_rate):
                point_array_sampled.append(point_array[i])

            point_array_sampled = np.array(point_array_sampled)

            point_array_all.append(point_array_sampled)

            continue
        elif filename.endswith(".lan"):
            path = os.path.join(directory, filename)
            #print("Processing:",path)
            landmarks = open(path)
            landmark_array = process_data(landmarks)

            land_mark_array_all.append(landmark_array)

            continue
            
    return point_array_all,land_mark_array_all

In [4]:
def calculate_transformation_matrix(original, aligned):
    transform_matrices=[]
    for i in range(0, len(original)):
        transform_matrix = np.zeros(shape=(3,4))
        
        transform_matrix[0][-1] = aligned[i][0][0] - original[i][0][0]
        transform_matrix[0][0] = 1
        
        transform_matrix[1][-1] = aligned[i][0][1] - original[i][0][1]
        transform_matrix[1][1] = 1
        
        transform_matrix[2][-1] = aligned[i][0][2] - original[i][0][2]
        transform_matrix[2][2] = 1
        
        transform_matrices.append(transform_matrix)
    
    return transform_matrices

In [5]:
def calculate_aligned_landmarks(landmarks, transform_matrices):
    updated_landmarks_all = []
    
    for i in range(0, len(landmarks)):
        updated_landmarks=[]
        
        if(len(landmarks[i])!=0):
            landmarks_1 = np.c_[landmarks[i],np.ones(len(landmarks[i]))]
                
            for j in range(0, len(landmarks[i])):
                updated_landmark=[]
                
                for row in transform_matrices[i]:
                    updated_landmark.append(np.dot(landmarks_1[j],row))
           
                
                updated_landmarks.append(updated_landmark)
            updated_landmarks_all.append(updated_landmarks)
        
    return np.array(updated_landmarks_all, dtype=object)

In [6]:
def length_width_calculator(foot_data):
    
    threshold = 0.5
    
    lengths = []
    widths = []
    
    for foot in foot_data:
        point_x = foot[:,0]
        point_z = foot[:,2]

        length = 0
        for point in range(int(min(point_x)),int(max(point_x))):
            indices_filtered = np.where((point_x>=point-threshold)&(point_x<=point+threshold))
            point_z_filtered = point_z[indices_filtered]
            
            if(len(point_z_filtered)>0):
                length_point = max(point_z_filtered) - min(point_z_filtered)
            if length_point>length:
                length = length_point
                
        lengths.append(length)
                
        width = 0
        for point in range(int(min(point_z)),int(max(point_z))):
            indices_filtered = np.where((point_z>=point-threshold)&(point_z<=point+threshold))
            point_x_filtered = point_x[indices_filtered]
            
            if(len(point_x_filtered)>0):
                width_point = max(point_x_filtered) - min(point_x_filtered)
            if width_point>width:
                width = width_point
        
        widths.append(width)
    
    return lengths,widths

In [7]:
def normalize(array):
    amin, amax = min(array), max(array)
    for i, val in enumerate(array):
        array[i] = (val-amin) / (amax-amin)
        
    return array

In [8]:
directory_aligned = "Datasets/Original Foot Data/Bounding Box Aligned/"
directory_processed = "Datasets/Original Foot Data/Processed/"

In [9]:
sample_rate = 1 #read by skipping given number of lines

In [10]:
aligned_points_all = read_directory(directory_aligned, sample_rate)[0]
original_points_all, original_landmarks_all = read_directory(directory_processed, sample_rate)

In [11]:
trans_mats = calculate_transformation_matrix(original_points_all, aligned_points_all)

In [12]:
aligned_landmarks_all = calculate_aligned_landmarks(original_landmarks_all,trans_mats)

In [13]:
point_x = aligned_points_all[0][:,0]
point_z = aligned_points_all[0][:,2]

In [14]:
threshold = 0.5
point = (max(point_x) + min(point_x))/2

In [15]:
#np.where(np.logical_and(a>=6, a<=10))

In [18]:
lengths,widths = length_width_calculator(aligned_points_all)
lengths[0]

234.82001495000003

In [46]:
fig1 = plt.figure()
ax = fig1.add_subplot()
ax.scatter(range(len(lengths)),lengths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
fig2 = plt.figure()
ax = fig2.add_subplot()
ax.scatter(range(len(lengths)),widths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
fig3 = plt.figure()
ax = fig3.add_subplot(111, projection='3d')
ax.scatter(range(len(lengths)),lengths,widths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
#kmeans = KMeans(n_clusters=10, random_state=0).fit()

In [50]:
lengths_normalized = normalize(lengths)

In [51]:
fig4 = plt.figure()
ax = fig4.add_subplot()
ax.scatter(range(len(lengths_normalized)),lengths_normalized)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
widths_normalized = normalize(widths)

In [53]:
fig5 = plt.figure()
ax = fig5.add_subplot()
ax.scatter(range(len(widths_normalized)),widths_normalized)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
fig6 = plt.figure()
ax = fig6.add_subplot(111, projection='3d')
ax.scatter(range(len(lengths_normalized)),lengths_normalized,widths_normalized)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
lengths

[0.5633829418731957,
 0.5635711078684601,
 0.5635705341463692,
 0.5635705341463702,
 0.5814290856424368,
 0.788961602457624,
 0.8083230430329795,
 0.7878333240146116,
 0.8081357376208511,
 0.6670173263481242,
 0.6757964422756996,
 0.6756085631414795,
 0.6756082764684171,
 0.6767361244318788,
 0.0607933660067963,
 0.4883780298289482,
 0.4883780298289482,
 0.508868178950903,
 0.4880018414569225,
 0.3188177385866957,
 0.337992304041491,
 0.3571664392047181,
 0.3759645294264433,
 0.37615212169961776,
 0.43104350309752193,
 0.16824442052606509,
 0.11335275207913398,
 0.16955928722859223,
 0.1741088291577285,
 0.17892111512651468,
 0.13139946938248404,
 0.17059362601050448,
 0.1695595740896388,
 0.601731068957109,
 0.31881730829512744,
 0.3199451564465722,
 0.3201338959759461,
 0.31750129414841727,
 0.3197575641733977,
 0.0,
 0.33780399480368584,
 0.3378045683377943,
 0.3378039948036853,
 0.3558504252459915,
 0.33780342108159483,
 0.9452874621773296,
 1.0,
 0.7329433767339524,
 0.75080092421